### === Purpose ===

The goal of this lab is to perform relation classification on a text where NER and Disambiguation were performed. For example, given a Wikipedia article:

    <Elvis_Presley>
    <Elvis_Presley> was an <United_States_of_America> singer and actor, married to <Priscilla_Presley>.

the goal is to predict the relation between the title entity and the others:

    <Elvis_Presley><nationality><United_States_of_America>
    <Elvis_Presley><spouse><Priscilla_Presley>

You will use a Language Model for this task, and make use of Constrained Decoding in order to make the predictions.

=== Provided Data ===

We provide
1. A preprocessed version of Wikipedia, wikipedia-ner.txt, which contains articles about disambiguated entities, whose content also went through NERC and Disambiguation.
2. A gold standard for the task, student-gold-standard.tsv, which contains triples <subject_entity> <object_entity> <relation>, that you will use to evaluate your method
3. a template for your code, relation_classification.py

### === Task ===

You will have two tasks in this lab.
The first will be to complete the function construct_trie, so that it constructs a trie for the (tokenized) list of relations given as input.
Your second task is to complete the function classify_relation in this file.
It receives as input (1) the title entity (subject), (2) the article content, (3) a trie.
It outputs a list of relations between the title entity and all the other disambiguated entities in the article content. It uses Language Models and Constrained Decoding.

In [ ]:
"""
Install necessary modules, run only once !
"""
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q accelerate

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from collections import defaultdict
import time
from tqdm import tqdm
import re
from typing import Dict, List

In [ ]:
"""
Loads a T5 LLM
"""
torch.cuda.empty_cache()
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
class WikipediaArticle:
  """ Represents a Wikipedia article. Do not modify. """
  def __init__(self, title, content):
    self.title_entity=title
    self.content=content

def wikipediaArticles(file):
  """ Yields the Wikipedia articles from a file. Do not modify. """
  article=[]
  title=None
  with open(file, "rt", encoding="utf=8") as inputFile:
    for line in inputFile:
      line=line.rstrip()
      if not title:
        title=line
        continue
      if not len(line) and title and len(article):
        yield WikipediaArticle(title, article[0])
        title=None
        article=[]
        continue
      article+=[line]

def clean(yagoEntity):
    """ Removes prefixes etc."""
    if yagoEntity.startswith('"'):
      return yagoEntity[1:-1]
    yagoEntity=yagoEntity[yagoEntity.find(':')+1:]
    return '<'+yagoEntity+'>'

In [ ]:
def run_evaluation():
  """Evaluation script, do not modify (unless you want to remove some prints).
  We use the f-05 measure, which gives more importance to precision: classifying entities correctly is more valued than finding all entities.
  """
  with open("student-gold-standard.tsv", "r", encoding="utf-8") as f:
    lines = f.readlines()
  gold_standard_dict = defaultdict(dict)
  for line in lines:
    title_entity, entity_id, relation = tuple(line.replace("\n","").split("\t"))
    gold_standard_dict[title_entity][entity_id] = relation
  gold_standard_dict = dict(gold_standard_dict)

  with open("results.tsv", "r", encoding="utf-8") as f:
    lines = f.readlines()
  predictions_dict = defaultdict(dict)
  for line in lines:
    title_entity, entity_id, relation = tuple(line.replace("\n","").split("\t"))
    predictions_dict[title_entity][entity_id] = relation

  true_pos = 0
  false_pos = 0
  false_neg = 0

  for title_entity in predictions_dict:
    for entity_id in predictions_dict[title_entity]:
      try:
        gold_yago_relation = gold_standard_dict[title_entity][entity_id]
      except KeyError: #should not happen
        continue
      if predictions_dict[title_entity][entity_id] == gold_yago_relation:
        true_pos += 1
      else:
        false_pos += 1
        if false_pos < 100:
          print("You classified the relation between", title_entity + " and " + entity_id, "wrong.", "Expected output: ", gold_yago_relation, ",given:", predictions_dict[title_entity][entity_id])

  for gold_title in gold_standard_dict: #do we really want to measure this? There are some entities that don't have a wikipedia article, so they count. Should they be removed from the gold standard?
    for entity_id in gold_standard_dict[gold_title]:
      try:
        predict_relation = predictions_dict[gold_title][entity_id]
      except KeyError:
        false_neg += 1
        if false_neg < 100:
          print("You did not classify the relation between", gold_title + " and " + entity_id +".")

  if true_pos + false_pos != 0:
    precision = float(true_pos) / (true_pos + false_pos)
  else:
    precision = 0.0

  if true_pos + false_neg != 0:
    recall = float(true_pos) / (true_pos + false_neg)
  else:
    recall = 0.0

  beta = 0.5

  if precision + recall != 0.0:
    f05 = (1 + beta * beta) * precision * recall / (beta * beta * precision + recall)
  else:
    f05 = 0.0

  print()
  print("Scores (scaled from 0 to 100)")
  print("Precision", precision*100)
  print("Recall", recall*100)
  print("F-0.5 Score", f05*100)

In [ ]:
def get_all_relations(file):
  with open(file, "r", encoding="utf-8") as f:
    lines = f.readlines()
  relations = set()
  for line in lines:
    title_entity, entity_id, relation = tuple(line.replace("\n","").split("\t"))
    relations.add(relation)
  return relations

In [ ]:
def prefix_allowed_tokens_fn(input_ids, trie):
  '''
  The function that handles constrained decoding.
  For the current generated text, returns the following allowed tokens. If nothing is allowed, return EOS token (ends the generation).
  This function is called at every generation step (every time a token is generated)
  DO NOT MODIFY (unless you're brave)
  '''
  model_output = input_ids.tolist()
  allowed_tokens = trie.get(model_output)
  if not allowed_tokens:
    return [tokenizer.eos_token_id]
  return allowed_tokens

In [ ]:
class Trie(object):
    def __init__(self, sequences: List[List[int]] = []):
        self.trie_dict = {}
        if sequences:
            for sequence in sequences:
                Trie._add_to_trie(sequence, self.trie_dict)

    def add(self, sequence: List[int]):
        Trie._add_to_trie(sequence, self.trie_dict)

    def get(self, prefix_sequence: List[int]):
        return Trie._get_from_trie(prefix_sequence, self.trie_dict)

    @staticmethod
    def _add_to_trie(sequence: List[int], trie_dict: Dict):
        if sequence:
            if sequence[0] not in trie_dict:
                trie_dict[sequence[0]] = {}
            Trie._add_to_trie(sequence[1:], trie_dict[sequence[0]])

    @staticmethod
    def _get_from_trie(prefix_sequence: List[int], trie_dict: Dict):
        if len(prefix_sequence) == 0:
            output = list(trie_dict.keys())
            return output
        elif prefix_sequence[0] in trie_dict:
            return Trie._get_from_trie(prefix_sequence[1:],trie_dict[prefix_sequence[0]])
        else:
            return []

    def __iter__(self):
        def _traverse(prefix_sequence, trie_dict):
            if trie_dict:
                for next_token in trie_dict:
                    yield from _traverse(prefix_sequence + [next_token], trie_dict[next_token])
            else:
                yield prefix_sequence

        return _traverse([], self.trie_dict)

    def __getitem__(self, value):
        return self.get(value)

In [ ]:
def construct_trie(relations: List[str], tokenizer):
    """
    Constructs a Trie for the list of relations and the tokenizer provided.
    Ensures token IDs are correctly encoded and includes <pad> at the start.
    """
    trie = Trie()
    for relation in relations:
        token_ids = tokenizer.encode(relation)
        trie.add([tokenizer.pad_token_id] + token_ids)  # Add <pad> at the start
    return trie

In [ ]:
def run_model(prompt, trie):
  """
  Runs the language model using constrained decoding
  Input:
  - Prompt (str)
  - prefix_allowed_tokens_fn
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  input_text = prompt
  inputs = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

  outputs = model.generate(inputs, max_new_tokens=20, do_sample=False, num_beams=5, temperature=None, top_p=None, pad_token_id=tokenizer.eos_token_id,
        prefix_allowed_tokens_fn=lambda _, input_ids: prefix_allowed_tokens_fn(input_ids, trie))
  return(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import re

def classify_relations(title_entity, page_content, trie):
    """ Disambiguates the entity name based on the Wikipedia article
    Returns a list of triples (title_entity, obj_entity, relation) or (title_entity, obj_entity, None)
    In the Wikipedia article, the title entity and the object entities are surrounded by "<" and ">"
    """
    triples = []

    # Full relation list
    full_relation_list = [
        '<birthPlace>', '<founder>', '<worksFor>', '<appearsIn>', '<musicBy>',
        '<actor>', '<productionCompany>', '<playsIn>', '<author>', '<director>',
        '<alumniOf>', '<homeLocation>', '<memberOf>', '<contentLocation>', '<location>',
        '<owns>', '<locationCreated>', '<manufacturer>', '<neighbors>', '<deathPlace>',
        '<deathDate>', '<children>', '<nationality>', '<award>', '<dateCreated>',
        '<about>', '<birthDate>'
    ]

    date_related_relations = ['<deathDate>', '<dateCreated>', '<birthDate>']

    # Extract entities from page_content (entities are surrounded by "<" and ">")
    entities = re.findall(r"<[^>]+>", page_content)

    # Filter out the title_entity itself
    obj_entities = [entity for entity in entities if entity != title_entity]

    # Generate relation for each obj_entity
    for obj_entity in obj_entities:
        # Detect if obj_entity is date-like or numeric
        is_date_like = bool(re.match(r"<\d{4}(-\d{2}){0,2}(T\d{2}:\d{2}:\d{2}Z)?>", obj_entity))

        # Use specific relation list if obj_entity is date-like
        relation_list = date_related_relations if is_date_like else full_relation_list
        relation_options = ', '.join(relation_list)

        # Create a T5 input prompt for each obj_entity
        prompt = f"""
        Context: {page_content}

        Task: Identify the relationship between the following entities.

        Entity 1: {title_entity}
        Entity 2: {obj_entity}

        Relation (choose from [{relation_options}]):

        """
        print(prompt)
        # Generate relation using the provided run_model function
        try:
            relation = run_model(prompt, trie)
            relation = relation.strip("<>").strip()  # Clean the relation to remove extra brackets if any

        except Exception as e:
            print(f"Error during generation for {title_entity} and {obj_entity}: {e}")
            print(f"Prompt was: {prompt}")
            relation = "unknown_relation"

        # Append the triple
        if relation:
            triples.append((title_entity, obj_entity, relation))
        else:
            triples.append((title_entity, obj_entity, None))

    return triples



In [ ]:
def run():
  relations = get_all_relations("student-gold-standard.tsv")
  trie = construct_trie(relations, tokenizer) #function to complete
  with open("results_4.tsv", 'a', encoding="utf-8") as output:
    start = time.time()
    for i, page in enumerate(wikipediaArticles("wikipedia-ner.txt")):
      print("  Processing",page.title_entity, i)
      result = classify_relations(page.title_entity, page.content, trie) #function to complete
      if result is not None:
        for subj, obj, rel in result:
          output.write(subj +"\t" + obj + "\t" + clean(rel) + "\n")
  end = time.time()
  print("done")
  print("execution time: ", end - start)
  print("number of articles: ", i)
run()

Streaming output truncated to the last 5000 lines.
        Entity 1: <Supergirl__u0028_Cir-El_u0029_>
        Entity 2: <Superman__u0028_comic_book_u0029_>

        Relation (choose from [<birthPlace>, <founder>, <worksFor>, <appearsIn>, <musicBy>, <actor>, <productionCompany>, <playsIn>, <author>, <director>, <alumniOf>, <homeLocation>, <memberOf>, <contentLocation>, <location>, <owns>, <locationCreated>, <manufacturer>, <neighbors>, <deathPlace>, <deathDate>, <children>, <nationality>, <award>, <dateCreated>, <about>, <birthDate>]):
  Processing <Power_Girl> 141

        Context: <Power_Girl>, also known as Kara Zor-L and Karen Starr, is a fictional DC Comics superheroine, making her first appearance in <All_Star_Comics> #58 (January/February 1976).Who’s Who in the DC Universe #18  Power Girl is the cousin of DC's flagship hero Superman, but from an alternative universe in the fictional multiverse in which DC Comics stories are set. <Power_Girl>, also known as Kara Zor-L and Karen St

In [ ]:
run_evaluation()

You classified the relation between <Ashok_Kumar__u0028_Indian_politician_u0029_> and <Indian_National_Congress> wrong. Expected output:  <memberOf> ,given: <alumniOf>
You classified the relation between <Ashok_Kumar__u0028_golfer_u0029_> and <Bihar> wrong. Expected output:  <birthPlace> ,given: <alumniOf>
You classified the relation between <Ashok_Kumar_Dogra> and <Indian_National_Congress> wrong. Expected output:  <memberOf> ,given: <alumniOf>
You classified the relation between <Ashok_Kumar> and <India> wrong. Expected output:  <nationality> ,given: <actor>
You classified the relation between <Ashok_Kumar_Garg> and <India> wrong. Expected output:  <nationality> ,given: <alumniOf>
You classified the relation between <Ashok_Kumar__u0028_British_politician_u0029_> and <Labour_Party__u0028_UK_u0029_> wrong. Expected output:  <memberOf> ,given: <alumniOf>
You classified the relation between <Ashok_Kumar__u0028_cinematographer_u0029_> and <Allahabad> wrong. Expected output:  <birthPlace> 